# Paso 1.
Realice la limpieza de los datasets:
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
 - Renombre la columna "default payment next month" a "default"
 - Remueva la columna "ID".

In [1]:
import pandas as pd

ruta_train = "../files/input/train_data.csv.zip"
ruta_test = "../files/input/test_data.csv.zip"
df_train = pd.read_csv(ruta_train, compression='zip')
df_test = pd.read_csv(ruta_test, compression='zip')

df_train = df_train.rename(columns={'default payment next month': 'default'})
df_test = df_test.rename(columns={'default payment next month': 'default'})

df_train = df_train.drop('ID', axis=1)
df_test = df_test.drop('ID', axis=1)

# print(df_train.shape)
# print(df_test.shape)
# df_train.head()

In [2]:
df_train = df_train.loc[(df_train['EDUCATION']!=0)]
df_test = df_test.loc[(df_test['EDUCATION']!=0)]
df_train = df_train.loc[(df_train['MARRIAGE']!=0)]
df_test = df_test.loc[(df_test['MARRIAGE']!=0)]

df_train.loc[df_train['EDUCATION'] > 4,'EDUCATION'] = 4
df_test.loc[df_test['EDUCATION'] > 4,'EDUCATION'] = 4

# print(df_train.shape)
# print(df_test.shape)

In [3]:
# Ni train ni test tienen valores nulos

# df_train[df_train.duplicated(keep='first')]

In [4]:
# df_test[df_test.duplicated(keep='first')]

In [5]:
df_train = df_train.drop_duplicates()
#df_test = df_test.drop_duplicates()

# print(df_train.shape)
# print(df_test.shape)

# Paso 2.
Divida los datasets en x_train, y_train, x_test, y_test.

In [6]:
X_train = df_train.drop(['default'], axis=1)
y_train = df_train['default']

X_test = df_test.drop(['default'], axis=1)
y_test = df_test['default']

In [7]:
X_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,310000,1,3,1,32,0,0,0,0,0,...,110375,84373,57779,14163,8295,6000,4000,3000,1000,2000
1,10000,2,3,1,49,-1,-1,-2,-1,2,...,-748,1690,1138,930,0,0,2828,0,182,0
2,50000,1,2,1,28,-1,-1,-1,0,-1,...,46257,45975,1300,43987,0,46257,2200,1300,43987,1386
3,80000,2,3,1,52,2,2,3,3,3,...,40101,40748,39816,40607,3700,1600,1600,0,1600,1600
4,270000,1,1,2,34,1,2,0,0,2,...,20924,22448,15490,17343,0,4000,2000,0,2000,2000


# Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método one-hot-encoding.
- Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
- Estandariza la matriz de entrada.
- Selecciona las K columnas mas relevantes de la matrix de entrada.
- Ajusta una maquina de vectores de soporte (svm).


In [8]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [9]:
semilla = 48

categorical = ['SEX','EDUCATION','MARRIAGE']
numerical = [col for col in X_train.columns if col not in categorical]

preprocessor = ColumnTransformer(
    transformers=[
        ('categories', OneHotEncoder(handle_unknown="ignore", dtype=int), categorical),
        ('scaler', StandardScaler(), numerical)
        ],
)
pca = PCA()
k = SelectKBest(score_func=f_classif)
svc = SVC(random_state=semilla)

steps = [
    ('preprocessor',  preprocessor),
    ('pca', pca),
    ('selectkbest', k),
    ('classifier', svc)
]

pipeline = Pipeline(steps)

# Paso 4.
Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. 
- Use la función de precision balanceada para medir la precisión del modelo.

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
param_grid = {
    'pca__n_components': [20, X_train.shape[1]],
    'selectkbest__k': [15],
    #'classifier__C': [],
    'classifier__gamma': [0.1],
    'classifier__kernel': [ 'rbf'],
    #'classifier__kernel': ['linear', 'rbf']
}


model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=-1,
    refit=True
)

model.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categories',
                                                                         OneHotEncoder(dtype=<class 'int'>,
                                                                                       handle_unknown='ignore'),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PAY_AMT4',
                                                                          'PAY_AMT5',
                                                                          'PAY_AMT6'])])),
                                       ('pca', PCA()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier', SVC(random_state=48))]),
             n_jobs=-1,
             param_grid={'classifier__gamma': [0.1],
                         'classifier__kernel': ['rbf'],
                         'pca__n_components': [20, 23],
                         'selectkbest__k': [15]},
             scoring='balanced_accuracy')

In [12]:
print('Mejores parámetros: ', model.best_params_)
print()
print("Precisión en train: ", model.score(X_train, y_train))
print("Precisión en test: ", model.score(X_test, y_test))

Mejores parámetros:  {'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'pca__n_components': 23, 'selectkbest__k': 15}

Precisión en train:  0.6653274836358736
Precisión en test:  0.6675753560270654


# Paso 5.
Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.


In [13]:
import pickle
import gzip
import os

os.makedirs('../files/models', exist_ok=True)

with gzip.open("../files/models/model.pkl.gz","wb") as file:
    pickle.dump(model, file)

# Paso 6.
Calcule las metricas para los conjuntos de entrenamiento y prueba:
- Precision
- Precision balanceada
- Recall
- f1-score 

Guardelas en el archivo files/output/metrics.json. 
- Cada fila del archivo es un diccionario con las metricas de un modelo.
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. 

Por ejemplo:

{'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}

{'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}


In [14]:
from sklearn.metrics import (
    precision_score,
    balanced_accuracy_score,
    recall_score, 
    f1_score,
)
import json

In [15]:
def calcular_metricas(modelo, X_dataset, y_dataset, tipo_dataset):
    y_pred = modelo.predict(X_dataset)
    metrics = {
    "type": "metrics", # Para el test
    'dataset': tipo_dataset,
    'precision': precision_score(y_dataset, y_pred),
    'balanced_accuracy': balanced_accuracy_score(y_dataset, y_pred),
    'recall': recall_score(y_dataset, y_pred),
    'f1_score': f1_score(y_dataset, y_pred)
    }
    return metrics

In [16]:
metricas_train = calcular_metricas(modelo= model, X_dataset=X_train, y_dataset=y_train, tipo_dataset='train')
metricas_test = calcular_metricas(modelo= model, X_dataset=X_test, y_dataset=y_test, tipo_dataset='test')

metricas = [metricas_train, metricas_test]

os.makedirs('../files/output', exist_ok=True)

with open('../files/output/metrics.json', 'w') as file:
    for metrica in metricas:
        file.write(json.dumps(metrica) + '\n')

# Paso 7.
Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. 
Guardelas en el archivo files/output/metrics.json. 
- Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba. 

Por ejemplo:

{'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}

{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}


In [17]:
from sklearn.metrics import confusion_matrix


In [18]:
def crear_cm(modelo, X_dataset, y_dataset, tipo_dataset):
    y_pred = modelo.predict(X_dataset)
    cm = confusion_matrix(y_dataset, y_pred)
    cm_dict = {
        'type': 'cm_matrix',
        'dataset': tipo_dataset,
        'true_0': {'predicted_0': int(cm[0, 0]), 'predicted_1': int(cm[0, 1])},
        'true_1': {'predicted_0': int(cm[1, 0]), 'predicted_1': int(cm[1, 1])}
    }
    return cm_dict

In [19]:
cm_train = crear_cm(modelo=model, X_dataset=X_train, y_dataset=y_train, tipo_dataset='train')
cm_test = crear_cm(modelo=model, X_dataset=X_test, y_dataset=y_test, tipo_dataset='test')

metricas_cm = [cm_train, cm_test]

with open('../files/output/metrics.json', 'a') as file:
    for metrica in metricas_cm:
        file.write(json.dumps(metrica) + '\n')